In [1]:
# import the libraries
import numpy as np
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from itertools import product
import matplotlib.pyplot as plt
# from fitter import Fitter, get_common_distributions, get_distributions

In [2]:
# import os

# # Set CUDA_LAUNCH_BLOCKING environment variable for better error messages
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [3]:
# import, shuffle, and see the data
ddf = dd.read_csv('D:/桌面/xiaoyiass/daima/train_data.csv')
shfl_ddf = ddf.sample(frac = 1, random_state = 42)
shfl_ddf.head()

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0001,reactivity_0002,reactivity_0003,...,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
25107,dfab8bf1dc9e,GGGAACGACUCGAGUAGAGUCGAAAAUGAACAAGUUGACGUUUCUC...,DMS_MaP,15k_DMS,1653,0.579,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31189,f8fab44626b9,GGGAACGACUCGAGUAGAGUCGAAAACCAACACACUCCUUCCCCCC...,2A3_MaP,DasLabBigLib_OneMil_15K_REP_2A3,122,0.346,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40167,2f4d8eb6a6f0,GGGAACGACUCGAGUAGAGUCGAAAAAGUUAUGUGCAUGUUGUAGA...,2A3_MaP,DasLabBigLib_OneMil_15K_REP_2A3,116,0.411,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17066,cd15884cb67a,GGGAACGACUCGAGUAGAGUCGAAAAGUGGAGUUCCUGUUGUAGAU...,DMS_MaP,15k_DMS,738,0.606,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31317,44af286d8277,GGGAACGACUCGAGUAGAGUCGAAAACUUAGAAGUGACAGGUGACA...,2A3_MaP,DasLabBigLib_OneMil_15K_REP_2A3,248,0.688,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dms_ddf = ddf.loc[ddf['experiment_type'] == "DMS_MaP"]
twoa3_ddf = ddf.loc[ddf['experiment_type'] == "2A3_MaP"]
dms_ddf.head()

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0001,reactivity_0002,reactivity_0003,...,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
15000,8cdfeef009ea,GGGAACGACUCGAGUAGAGUCGAAAAACGUUGAUAUGGAUUUACUC...,DMS_MaP,15k_DMS,1668,0.972,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15001,51e61fbde94d,GGGAACGACUCGAGUAGAGUCGAAAAACAUUGAUAUGGAUUUACUC...,DMS_MaP,15k_DMS,2421,0.167,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15002,25ce8d5109cd,GGGAACGACUCGAGUAGAGUCGAAAAACCUUGAUAUGGAUUUACUC...,DMS_MaP,15k_DMS,1964,1.848,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15003,07dcfb6d1965,GGGAACGACUCGAGUAGAGUCGAAAAACUUUGAUAUGGAUUUACUC...,DMS_MaP,15k_DMS,45863,9.291,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15004,e561cc042a4c,GGGAACGACUCGAGUAGAGUCGAAAAACGAUGAUAUGGAUUUACUC...,DMS_MaP,15k_DMS,6219,3.210,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Get the shape of the Dask DataFrame
shape = dms_ddf.shape

# Extract the number of rows and columns from the shape tuple
num_rows, num_columns = shape[0].compute(), shape[1]

# Print the size of the Dask DataFrame
print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 821840
Number of Columns: 419


In [6]:
bases={'A':0, 'C':1, 'G':2, 'T':3 }

def one_hot(string):

    res = np.zeros((5, 206),
                   dtype=np.float32)

    for j in range(len(string)):
        if string[j] in bases: # bases can be 'N' signifying missing: this corresponds to all 0 in the encoding
            res[ bases[ string[j] ], j ]= 1.
    for j in range(len(string),206):
        res[4, j]= 1.
    return res
# one_hot(seqs[0])

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BedPeaksDataset(torch.utils.data.IterableDataset):

    def __init__(self, seq, reactivities):
        super(BedPeaksDataset, self).__init__()
        self.seq = seq
        self.reactivities = reactivities

    def __iter__(self):
        for i in range(len(self.seq)):
            yield(one_hot(self.seq[i]), self.reactivities[i]) # positive example

# train_dataset = BedPeaksDataset(seqs, reactivities)
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=10, num_workers = 0)

In [8]:
def run_one_epoch(train_flag, dataloader, cnn_1d, optimizer, device="cuda"):

    torch.set_grad_enabled(train_flag)
    cnn_1d.train() if train_flag else cnn_1d.eval()

    losses = []
    accuracies = []

    for (x,y) in dataloader: # collection of tuples with iterator
        x = x.float()
        y = y.float()
        (x, y) = ( x.to(device), y.to(device) ) # transfer data to GPU

        output = cnn_1d(x,y) # forward pass
        output = output.squeeze() # remove spurious channel dimension
        loss = F.mse_loss(output, y).float()

        if train_flag:
            loss.backward() # back propagation
            optimizer.step()
            optimizer.zero_grad()

        losses.append(loss.detach().cpu().numpy())
        # accuracy = torch.mean( ( (output > 0.0) == (y > 0.5) ).float() ) # output is in logit space so threshold is 0.
        # accuracies.append(accuracy.detach().cpu().numpy())

    return( np.mean(losses))

In [9]:
# F.mse_loss(torch.tensor([5., 5., 5.]), torch.tensor([6., 6., 6.]))

In [10]:
def train_model(cnn_1d, train_dataloader, test_dataloader, epochs=100, patience=10, verbose = True, lr = 0.001, weight_decay = 0):
    """
    Train a 1D CNN model and record accuracy metrics.
    """
    # Move the model to the GPU here to make it runs there, and set "device" as above
    # TODO CODE
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_1d.to(device)

    # 1. Make new BedPeakDataset and DataLoader objects for both training and validation data.
    # TODO CODE
#     train_dataset = BedPeaksDataset(train_data, genome, cnn_1d.seq_len)
#     train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=10, num_workers = 0)
    # validation_dataset = BedPeaksDataset(validation_data, genome, cnn_1d.seq_len)
    # validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1000)

    # 2. Instantiates an optimizer for the model.
    # TODO CODE
    optimizer = torch.optim.Adam(cnn_1d.parameters(), amsgrad=True, lr = lr, weight_decay = weight_decay)

    # 3. Run the training loop with early stopping.
    # TODO CODE
    train_losses = []
    test_losses = []
    # patience_counter = patience
    best_test_loss = np.inf
    check_point_filename = 'cnn_1d_checkpoint.pt' # to save the best model fit to date
    for epoch in range(epochs):
        start_time = timeit.default_timer()
        train_loss = run_one_epoch(True, train_dataloader, cnn_1d, optimizer, device)
        test_loss = run_one_epoch(False, test_dataloader, cnn_1d, optimizer, device)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        # train_accs.append(train_acc)
        # val_accs.append(val_acc)
        if test_loss < best_test_loss:
            torch.save(cnn_1d.state_dict(), check_point_filename)
            best_test_loss = test_loss
            patience_counter = patience
        else:
            patience_counter -= 1
            if patience_counter <= 0:
                cnn_1d.load_state_dict(torch.load(check_point_filename)) # recover the best model so far
                break
        elapsed = float(timeit.default_timer() - start_time)
        print("Epoch {} took {:.2f}s. Train loss: {:.4f}., Test loss: {:.4f}. Pariance: {}".format(epoch+1, elapsed, train_loss, test_loss, patience))

    # 4. Return the fitted model (not strictly necessary since this happens "in place"), train and validation accuracies.
    # TODO CODE
    return(cnn_1d, train_losses, test_losses)

In [11]:
! pip install dask_ml

Looking in indexes: https://pypi.org/simple/


In [12]:
# apply SN-filter
from dask_ml.model_selection import train_test_split
df_sn = ddf[ddf["SN_filter"]==1]

# split into 2A3 MaP and DMS MaP datasets
df_2A3 = df_sn[df_sn["experiment_type"]=="2A3_MaP"]
df_DMS = df_sn[df_sn["experiment_type"]=="DMS_MaP"]

# split into train and test
X_2A3 = df_2A3["sequence"]
y_2A3 = df_2A3.loc[:, df_2A3.columns.str.fullmatch("reactivity_\d\d\d\d")]
X_2A3_train, X_2A3_test, y_2A3_train, y_2A3_test = train_test_split(X_2A3, y_2A3, test_size=0.2, shuffle=True, blockwise=True, random_state=42)

X_DMS = df_DMS["sequence"]
y_DMS = df_DMS.loc[:, df_DMS.columns.str.fullmatch("reactivity_\d\d\d\d")]
X_DMS_train, X_DMS_test, y_DMS_train, y_DMS_test = train_test_split(X_DMS, y_DMS, test_size=0.2, shuffle=True, blockwise=True, random_state=42)

In [13]:
def df_toArray(ddf1, ddf2):
    subset_columns = []
    for i in range(206):
        subset_columns.append("reactivity_0"+str(i+1).zfill(3))

    # Use .to_dask_array() to convert the subset of the DataFrame to a Dask Array
    # subset = small_set[subset_columns]

    # Compute the subset of the Dask DataFrame and convert it to a Pandas DataFrame
    reactivities = ddf2.compute().to_numpy()

    row_means = np.nanmean(reactivities, axis=1)

    # Iterate over each element and replace NaN with the row mean
    for i, row in enumerate(reactivities):
        mask = np.isnan(row)
        reactivities[i, mask] = row_means[i]

    reactivities

    seqs = ddf1.compute().tolist()
    return seqs, reactivities
    
# df_toArray(X_2A3_train, y_2A3_train)

In [14]:
seqs, reactivities = df_toArray(X_2A3_train, y_2A3_train)
train_dataset = BedPeaksDataset(seqs, reactivities)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=100, num_workers = 0)

seqs, reactivities = df_toArray(X_2A3_test, y_2A3_test)
test_dataset = BedPeaksDataset(seqs, reactivities)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=100, num_workers = 0)

In [15]:
# import timeit
# my_cnn1d_1 = CNN_1d()
# # print(my_cnn1d.seq_le
# my_cnn1d_1 = my_cnn1d_1.float()
# my_cnn1d_1, train_losses, test_losses = train_model(my_cnn1d_1, train_dataloader, test_dataloader, lr = 0.001, weight_decay = 0)

In [16]:
# device = 'cuda'
# outputs = []
# expected = []
# for (x,y) in train_dataloader: # iterate over batches
#     # print(y)
#     if torch.cuda.is_available():
#         x = x.to(device)
#     output = my_cnn1d_1(x).squeeze() # your awesome model here!
#     output = torch.sigmoid(output)
#     output_np = output.detach().cpu().numpy()
#     outputs.append(output_np)
#     expected.append(y.numpy())
# output_np = np.concatenate(outputs)
# expected_np = np.concatenate(expected)

# print(output_np)
# print(expected_np)

# output_np = output_np.clip(0, 1)
# expected_np = expected_np.clip(0,1)

# mae1 = np.mean(np.abs(output_np - expected_np))
# mae1

In [17]:
# seqs, reactivities = df_toArray(X_DMS_train, y_DMS_train)
# train_dataset = BedPeaksDataset(seqs, reactivities)
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, num_workers = 0)

# seqs, reactivities = df_toArray(X_DMS_test, y_DMS_train)
# test_dataset = BedPeaksDataset(seqs, reactivities)
# test_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, num_workers = 0)

# my_cnn1d_2 = CNN_1d()
# # print(my_cnn1d.seq_le
# my_cnn1d_2 = my_cnn1d_2.float()
# my_cnn1d_2, train_losses, test_losses = train_model(my_cnn1d_2, train_dataloader, test_dataloader, lr = 0.001, weight_decay = 0)

In [18]:
# device = 'cuda'
# outputs = []
# expected = []
# for (x,y) in train_dataloader: # iterate over batches
#     # print(y)
#     if torch.cuda.is_available():
#         x = x.to(device)
#     output = my_cnn1d_2(x).squeeze() # your awesome model here!
#     output = torch.sigmoid(output)
#     output_np = output.detach().cpu().numpy()
#     outputs.append(output_np)
#     expected.append(y.numpy())
# output_np = np.concatenate(outputs)
# expected_np = np.concatenate(expected)

# print(output_np)
# print(expected_np)

# output_np = output_np.clip(0, 1)
# expected_np = expected_np.clip(0,1)

# mae2 = np.mean(np.abs(output_np - expected_np))
# mae2

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [20]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import math
# import copy

# def clones(module, N):
#     return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# class TransformerModel(nn.Module):
#     def __init__(self, input_dim=5, d_model=512, nhead=8, num_encoder_layers=6, dim_feedforward=2048, n_output_channels = 206, dropout=0.2):
#         super(TransformerModel, self).__init__()

#         # Embedding layer that will transform input to match d_model size
#         self.embedding = nn.Linear(input_dim, d_model)

#         # Positional Encoding
#         self.pos_encoder = PositionalEncoding(d_model, dropout)

#         # Transformer Encoder Layers
#         self_attn = MultiHeadAttention(d_model, nhead)
#         ff = PositionWiseFeedForward(d_model, 206)
#         self.encoder_layers = clones(EncoderLayer(d_model, self_attn, ff, dropout), num_encoder_layers)
#         self.norm = LayerNorm(d_model)

#         # Decoder layer to bring the output to the desired n_output_channels
#         self.decoder = nn.Linear(d_model * 206, n_output_channels)  # Adjust the multiplication factor as per your input sequence length

#     def forward(self, src):
#         # Reshape input to [batch_size, seq_len, input_dim]
#         src = src.view(src.size(0), -1, 5)  # Assuming src has shape [batch_size, 5, 206]
        
#         # Pass through the embedding layer
#         src = self.embedding(src)

#         # Add positional encoding
#         src = self.pos_encoder(src)

#         # Pass through the Transformer Encoder
#         for layer in self.encoder_layers:
#             src = layer(src)

#         src = self.norm(src)
#         src = src.view(src.size(0), -1)

#         # Pass through the decoder
#         output = self.decoder(src)
#         return output

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, dropout=0.1, max_len=5000):
#         super(PositionalEncoding, self).__init__()
#         self.dropout = nn.Dropout(p=dropout)

#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0).transpose(0, 1)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         x = x + self.pe[:x.size(0), :]
#         return self.dropout(x)

# # ... Include the other classes such as LayerNorm, PositionalEncoding, MultiHeadedAttention, etc.

# # Example instantiation of the model
# # n_output_channels = 10  # Modify as needed
# # transformer = TransformerModel(n_output_channels=n_output_channels)


In [21]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import math

# class TransformerModel(nn.Module):
#     def __init__(self, input_dim=5, d_model=512, nhead=8, num_encoder_layers=6, dim_feedforward=2048, n_output_channels = 206, dropout=0.2):
#         super(TransformerModel, self).__init__()

#         # Embedding layer that will transform input to match d_model size
#         self.embedding = nn.Linear(input_dim, d_model)

#         # Positional Encoding
#         self.pos_encoder = PositionalEncoding(d_model, dropout)

#         # Transformer Encoder
#         encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
#         self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_encoder_layers)

#         # Decoder layer to bring the output to the desired n_output_channels
#         self.decoder = nn.Linear(d_model, n_output_channels)

#     def forward(self, src):
#         # Reshape input to [batch_size, seq_len, input_dim]
#         src = src.view(src.size(0), -1, 5)  # Assuming src has shape [batch_size, 5, 206]
        
#         # Pass through the embedding layer
#         src = self.embedding(src)

#         # Add positional encoding
#         src = self.pos_encoder(src)

#         # Pass through the Transformer Encoder
#         output = self.transformer_encoder(src)

#         # Flatten the output for the linear layer
#         output = output.view(output.size(0), -1)

#         # Pass through the decoder
#         output = self.decoder(output)
#         return output

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, dropout=0.1, max_len=5000):
#         super(PositionalEncoding, self).__init__()
#         self.dropout = nn.Dropout(p=dropout)

#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0).transpose(0, 1)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         x = x + self.pe[:x.size(0), :]
#         return self.dropout(x)

# # # Example instantiation of the model
# # n_output_channels = 206  # Modify as needed
# # transformer = TransformerModel(n_output_channels=n_output_channels)


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class TransformerModel(nn.Module):
    def __init__(self, input_dim=5, d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, n_output_channels=10, dropout=0.2):
        super(TransformerModel, self).__init__()
        self.input_dim = input_dim
        # Embedding layers that will transform input to match d_model size
        self.embedding = nn.Linear(input_dim, d_model)
        self.target_embedding = nn.Linear(1, d_model)

        # Positional Encoding
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.pos_decoder = PositionalEncoding(d_model, dropout)

        # Transformer Encoder
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_encoder_layers)

        # Transformer Decoder
        decoder_layers = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layers, num_layers=num_decoder_layers)

        # Decoder layer to bring the output to the desired n_output_channels
        self.decoder = nn.Linear(d_model, n_output_channels)

    def forward(self, src, tgt):
        # Reshape input to [seq_len, batch_size, input_dim]
#         print(src.shape)
#         print(tgt.shape)
        src = src.view(-1, src.size(0), 5)  # Reshaped to [src_seq_len, batch_size, input_dim]
        tgt = tgt.view(-1, tgt.size(0), 1)  # Reshaped to [tgt_seq_len, batch_size, input_dim]
        
        # Pass through the embedding layers
        src = self.embedding(src)
        tgt = self.target_embedding(tgt)

        # Add positional encoding
        src = self.pos_encoder(src)
        tgt = self.pos_decoder(tgt)

        # Pass through the Transformer Encoder
        memory = self.transformer_encoder(src)

        # Pass through the Transformer Decoder
        output = self.transformer_decoder(tgt, memory)

        output = torch.mean(output, dim=0)  # Now the shape should be [batch_size, d_model]
    
        # Now reshape it to match the linear layer's input expectation
        output = output.view(output.size(0), -1)  # Shape should be [batch_size, d_model]

        # Pass through the decoder
        output = self.decoder(output)  # This should work as expected now
#         print(output.shape)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# # Example instantiation of the model
# n_output_channels = 10  # Modify as needed
# transformer = TransformerModel(n_output_channels=n_output_channels)


In [23]:
import timeit
# my_cnn1d_1 = CNN_1d()
# print(my_cnn1d.seq_le
# my_cnn1d_1 = my_cnn1d_1.float()
transformer = TransformerModel(input_dim= 5, d_model= 512, nhead= 8, num_encoder_layers = 2, dim_feedforward = 2048, n_output_channels = 206)
transformer, train_losses, test_losses = train_model(transformer, train_dataloader, test_dataloader, lr = 0.001, weight_decay = 0)

c:\Python311\Lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1 took 384.68s. Train loss: 0.1732., Test loss: 0.1844. Pariance: 10
Epoch 2 took 383.59s. Train loss: 0.1709., Test loss: 0.1841. Pariance: 10
Epoch 3 took 383.66s. Train loss: 0.1707., Test loss: 0.1839. Pariance: 10
Epoch 4 took 383.61s. Train loss: 0.1707., Test loss: 0.1839. Pariance: 10
Epoch 5 took 375.41s. Train loss: 0.1708., Test loss: 0.1839. Pariance: 10
Epoch 6 took 379.46s. Train loss: 0.1709., Test loss: 0.1840. Pariance: 10
Epoch 7 took 383.76s. Train loss: 0.1709., Test loss: 0.1839. Pariance: 10
Epoch 8 took 383.79s. Train loss: 0.1709., Test loss: 0.1839. Pariance: 10
Epoch 9 took 383.70s. Train loss: 0.1710., Test loss: 0.1840. Pariance: 10
Epoch 10 took 383.91s. Train loss: 0.1711., Test loss: 0.1840. Pariance: 10
Epoch 11 took 383.87s. Train loss: 0.1711., Test loss: 0.1839. Pariance: 10
Epoch 12 took 383.77s. Train loss: 0.1711., Test loss: 0.1838. Pariance: 10


KeyboardInterrupt: 

In [24]:
device = 'cuda'
outputs = []
expected = []
transformer.eval()

for (x, y) in test_dataloader:  # iterate over batches
    if torch.cuda.is_available():
        y = y.to(device).float()
        x = x.to(device).float()

    output = transformer(x, y).squeeze()  # your awesome model here!

    output_np = output.detach().cpu().numpy()
    outputs.append(output_np)
    # Move y to cpu before converting to numpy array
    expected_np = y.cpu().numpy()  # Move to CPU for NumPy conversion
    expected.append(expected_np)

# Concatenate all the outputs and expected results
output_np = np.concatenate(outputs)
expected_np = np.concatenate(expected)

print(output_np)
print(expected_np)

# Clip the values to be between 0 and 1
output_np = output_np.clip(0, 1)
expected_np = expected_np.clip(0, 1)

# Calculate the Mean Absolute Error
mae1 = np.mean(np.abs(output_np - expected_np))
mae1


[[0.43928346 0.45013425 0.4399463  ... 0.44021487 0.45080703 0.44303825]
 [0.43928346 0.45013422 0.43994626 ... 0.44021487 0.45080703 0.44303823]
 [0.43928352 0.45013422 0.43994626 ... 0.44021484 0.4508071  0.44303828]
 ...
 [0.43928352 0.45013425 0.43994632 ... 0.44021487 0.4508071  0.4430383 ]
 [0.43928352 0.45013425 0.4399463  ... 0.44021487 0.45080703 0.44303825]
 [0.4392835  0.45013428 0.43994632 ... 0.44021484 0.4508071  0.44303828]]
[[0.40836895 0.40836895 0.40836895 ... 0.40836895 0.40836895 0.40836895]
 [0.3389806  0.3389806  0.3389806  ... 0.3389806  0.3389806  0.3389806 ]
 [0.49919    0.49919    0.49919    ... 0.49919    0.49919    0.49919   ]
 ...
 [0.33333945 0.33333945 0.33333945 ... 0.33333945 0.33333945 0.33333945]
 [0.3354348  0.3354348  0.3354348  ... 0.3354348  0.3354348  0.3354348 ]
 [0.25894204 0.25894204 0.25894204 ... 0.25894204 0.25894204 0.25894204]]


0.20098984

In [ ]:
seqs, reactivities = df_toArray(X_DMS_train, y_DMS_train)
train_dataset = BedPeaksDataset(seqs, reactivities)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=100, num_workers = 0)

seqs, reactivities = df_toArray(X_2A3_test, y_2A3_test)
test_dataset = BedPeaksDataset(seqs, reactivities)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=100, num_workers = 0)

In [ ]:
import timeit
# my_cnn1d_1 = CNN_1d()
# print(my_cnn1d.seq_le
# my_cnn1d_1 = my_cnn1d_1.float()
transformer = TransformerModel(input_dim= 5, d_model= 512, nhead= 8, num_encoder_layers = 2, dim_feedforward = 2048, n_output_channels = 206)
transformer, train_losses, test_losses = train_model(transformer, train_dataloader, test_dataloader, lr = 0.001, weight_decay = 0)

RuntimeError: CUDA error: the launch timed out and was terminated
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
device = 'cuda'
outputs = []
expected = []
transformer.eval()

for (x, y) in test_dataloader:  # iterate over batches
    if torch.cuda.is_available():
        y = y.to(device).float()
        x = x.to(device).float()

    output = transformer(x, y).squeeze()  # your awesome model here!
    #output = torch.sigmoid(output)
    output_np = output.detach().cpu().numpy()
    outputs.append(output_np)
    # Move y to cpu before converting to numpy array
    expected_np = y.cpu().numpy()  # Move to CPU for NumPy conversion
    expected.append(expected_np)

# Concatenate all the outputs and expected results
output_np = np.concatenate(outputs)
expected_np = np.concatenate(expected)

print(output_np)
print(expected_np)

# Clip the values to be between 0 and 1
output_np = output_np.clip(0, 1)
expected_np = expected_np.clip(0, 1)

# Calculate the Mean Absolute Error
mae1 = np.mean(np.abs(output_np - expected_np))
mae1

In [ ]:
# Training 
criterion = F.mse_loss(output, y).float()
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

NameError: name 'src_data' is not defined

In [ ]:
first_10_rows_ddf = dms_ddf.head(10)
seqs1 = dms_ddf['sequence'].compute().tolist()
seqs2 = twoa3_ddf['sequence'].compute().tolist()

In [ ]:
for i in range(len(seqs1)):
    if len(seqs1[i])!= 170 or len(seqs1[i])!= 177:
        print(len(seqs1[i]))

The unigram, bigram, etc. may have different reactivity distributions for their nucleotides.

Let's examine the reactivity distributions for the nucleotides of unigram, bigram, etc.

In [ ]:
# Given a row with a sequence and a subsequence,
# returns a list of the reactivities of all
# instances of the subsequence in the sequence
# Example:
#    getReactOfSubseqForRow(row, 'AUG') ->  [[-0.027, 0.429, 0.817], [-0.079, -0.014, 0.328]]
def getReactOfSubseqForRow(row, subseq):
    
    react = []
    
    for i in range(len(row['sequence']) - len(subseq)):
        
        if row['sequence'][i:i+len(subseq)] == subseq:
            
            react_curr = []
            for j in range(i,i+len(subseq)):
                react_curr.append(row['reactivity_' + f'{j+1:04}'])
            
            if not np.isnan(react_curr).any():
                react.append(react_curr)
    
    return np.array(react)

# Given a dataframe with sequences and a subsequence,
# returns a dataframe of the reactivities of all
# instances of the subsequence in the sequence
# Example:
#    getReactOfSubseqForRow(df, 'AUG') ->  
#        ###############################
#        # react_A # react_U # react_G #
#        ###############################
#        #   1.221 #   1.031 #   1.098 #
#        #   1.272 #   1.453 #   1.007 #
#        #   4.446 #   0.000 #   0.000 #
#        ###############################
def getReactOfSubseqForFrame(df, subseq):
    return df.apply(
                getReactOfSubseqForRow,
                subseq=subseq,
                axis=1
            ).explode().dropna().to_frame().apply(
                lambda row: row[0],
                axis=1,
                result_type='expand'
            ).rename(columns = {
                i: f'react_{val}_{i}'
                for i, val in enumerate(subseq)
            })

In [ ]:
pdf_sn1 = shfl_ddf[shfl_ddf["SN_filter"] == 1]
pdf_2a3 = pdf_sn1[pdf_sn1["experiment_type"] == "2A3_MaP"].head(n=1000, compute=True)

In [ ]:
for i in range(1, 3):
    for subseq in product("AUGC", repeat = i):
        getReactOfSubseqForFrame(pdf_2a3, "".join(subseq)).hist(bins=20)
        plt.show()
        plt.close()

The histograms tend to be right skewed and peak close to 0.

In the competition, the goal is not to predict reactivity, but reactivity (bounded), which is reactivity bounded between 0 and 1.

Let's examine reactivity bounded.

In [ ]:
for i in range(1, 3):
    for subseq in product("AUGC", repeat = i):
        getReactOfSubseqForFrame(pdf_2a3, "".join(subseq)).clip(0,1).hist(bins=20)
        plt.show()
        plt.close()

The histograms often have 2 peaks, one at 0 and one at 1.

The histograms seem to be very similar across subsequences.

Let's examine the distribution for subsequences of length 1. I am using a tool from [here](https://medium.com/the-researchers-guide/finding-the-best-distribution-that-fits-your-data-using-pythons-fitter-library-319a5a0972e9).

In [ ]:
for b in "AUGC":
    react = getReactOfSubseqForFrame(pdf_2a3, b).iloc[:, 0].values
    f = Fitter(react)
    f.fit(progress=True)
    print(f.summary())
    print(b)
    plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fe343e39-d2c0-4296-915d-091d9a42752d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>